In [1]:
!pip install openai
!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

# OpenAI Functions

Ok, let's first understand how [OpenAI](https://openai.com/) the company behind ChatGPT, allows for these function call implementations in its API.

OpenAI implemented a [function calling API](https://platform.openai.com/docs/guides/function-calling) which is a standard way to connect their models to outside tools like in the very simple example we did above.

According to their [official documentation](https://platform.openai.com/docs/guides/function-calling#:~:text=The%20basic%20sequence,to%20the%20user.) the sequence of steps for function calling is as follows:

<img src="./assets-resources/openai-function-calling-diagram.png" width=300px;>

Below is an example taken from their official documentation:

In [1]:
from openai import OpenAI
import json

client = OpenAI()

Let's look at how our previous model with those three simple functions: `create_directory()`, `create_file()`, and `list_files()` would be implemented using OpenAI's function calling approach:

In [2]:
import json
import subprocess

def create_directory(directory_name):
    """Function that creates a directory given a directory name."""""
    subprocess.run(["mkdir", directory_name])
    return json.dumps({"directory_name": directory_name})


tool_create_directory = {
    "type": "function",
    "function": {
        "name": "create_directory",
        "description": "Create a directory given a directory name.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory_name": {
                    "type": "string",
                    "description": "The name of the directory to create.",
                }
            },
            "required": ["directory_name"],
        },
    },
}

tools = [tool_create_directory]    

In [3]:
def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

In [4]:
import json

def run_terminal_task():
    messages = [{"role": "user", "content": "Create a folder called 'lucas-the-agent-master'."}]
    tools = [tool_create_directory]  
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "create_directory": create_directory,
        }
        messages.append(response_message)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                directory_name=function_args.get("directory_name"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return second_response

output = run_terminal_task()
output

ChatCompletion(id='chatcmpl-A3P5r1KyUO4wwGS92Jecu0m1YC6py', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The folder 'lucas-the-agent-master' has been successfully created.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725374807, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f33667828e', usage=CompletionUsage(completion_tokens=14, prompt_tokens=56, total_tokens=70))

In [5]:
output.choices[0].message.content

"The folder 'lucas-the-agent-master' has been successfully created."

In [6]:
!ls -d */

assets-resources/       images/                 my-app/
docs/                   lucas-the-agent-master/ pages/


Great! We implemented openai function calling for creating directories! We could evolve this approach but let's stop for now.

See more info on these examples from OpenAI's [official cookbook](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).